---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevincrotty.rice.edu/)
    
Jones Graduate School of Business, Rice University

---


# U.S. STOCK MARKET RETURN

In [20]:
from pandas_datareader import DataReader as pdr

mkt = pdr('F-F_Research_Data_Factors','famafrench',start=1926)[1]/100
mkt = mkt['Mkt-RF'] + mkt.RF
mkt.index.name = "year"
mkt.name = "market"
mkt.index = mkt.index.astype(str).astype(int)
mkt.tail()

year
2018   -0.0512
2019    0.3043
2020    0.2411
2021    0.2360
2022   -0.2017
Name: market, dtype: float64

# RETURNS FROM YAHOO FINANCE

In [21]:
import yfinance as yf

ticker = input("Input a ticker")
price = yf.download(ticker, start="1970-01-01")["Adj Close"]
price = price.resample("Y").last()
ret = price.pct_change()[:-1].dropna()
ret.index = ret.index.map(lambda x: x.year)
ret.index.name = "year"
ret.name = ticker
ret.tail()

[*********************100%***********************]  1 of 1 completed


year
2018   -0.097474
2019    0.152736
2020   -0.259521
2021    0.462413
2022    0.584624
Name: cvx, dtype: float64

# YAHOO RETURN OR MARKET

In [22]:
flag = int(input("input 0 for market or 1 for yahoo return"))
data = ret if flag else mkt
name = f"{ticker.upper()}" if flag else "Market"

# BEST AND WORST PERIODS

In [23]:
geomean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   endyear    77 non-null     int64  
 1   geomean    77 non-null     float64
 2   startyear  77 non-null     int32  
dtypes: float64(1), int32(1), int64(1)
memory usage: 1.6 KB


In [24]:
years = int(input("input desired length of time period in years "))

compound = data.rolling(years).apply(lambda x: (1 + x).prod())
bestyear = compound.idxmax()
worstyear = compound.idxmin()

geomean = compound.dropna()**(1/years) - 1
geomean = geomean.reset_index()
geomean.columns = ["endyear", "geomean"]
geomean["startyear"] = geomean.endyear.astype(int) - years + 1

best = data.loc[(bestyear - years + 1) : bestyear]
best = (1 + best).cumprod()
best = best.reset_index()
best.columns = ["year", "compounded_return"]
best["startyear"] = best.year.iloc[0]

worst = data.loc[(worstyear - years + 1) : worstyear]
worst = (1 + worst).cumprod()
worst = worst.reset_index()
worst.columns = ["year", "compounded_return"]
worst["startyear"] = worst.year.iloc[0]

# FIGURE 1

In [25]:
import plotly.graph_objects as go

trace = go.Scatter(
    x=best.year,
    y=best.compounded_return,
    text=best.startyear,
    hovertemplate="%{text}-%{x}<br>$%{y:.2f}",
    mode="lines+markers"
)
fig = go.Figure(trace)
fig.update_layout(
    title = f"Best {years} Year Period",
    xaxis_title="Year",
    yaxis_title=f"{name} Accumulation from $1",
    yaxis_tickprefix="$",
    yaxis_tickformat=".2f",
    template="plotly_white"
)
fig.show()

# FIGURE 2

In [26]:
trace = go.Scatter(
    x=worst.year,
    y=worst.compounded_return,
    text=worst.startyear,
    hovertemplate="%{text}-%{x}<br>$%{y:.2f}",
    mode="lines+markers"
)
fig = go.Figure(trace)
fig.update_layout(
    title = f"Worst {years} Year Period",
    xaxis_title="Year",
    yaxis_title=f"{name} Accumulation from $1",
    yaxis_tickprefix="$",
    yaxis_tickformat=".2f",
    template="plotly_white"
)
fig.show()

# FIGURE 3

In [29]:
trace = go.Scatter(
    x=geomean.endyear,
    y=geomean.geomean,
    text=geomean.startyear,
    hovertemplate="%{text}-%{x}<br>%{y:.2%}",
    mode="lines+markers"
)
fig = go.Figure(trace)
fig.update_layout(
    title = f"{years} Year {name} Geometric Average Returns",
    xaxis_title="Ending Year",
    yaxis_title="Geometric Average Return",
    yaxis_tickformat=".2f",
    template="plotly_white"
)
fig.show()

# FIGURE 4

In [33]:
trace = go.Box(
    x = geomean.geomean,
    name="Geometric Means"
)
fig = go.Figure(trace)
fig.update_layout(
    template="plotly_white",
)
fig.show()
